# Модель выявления подозрительных платежей

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [22]:
hist_trx = pd.read_csv('C:\\Users\\firek\\Desktop\\hw1\\hist_trx.csv', index_col=0)
hits0712 = pd.read_csv('C:\\Users\\firek\\Desktop\\hw1\\hits0712.csv', index_col=0)

In [23]:
hist_trx.shape

(562490, 10)

Берём предполагаемого мошенника и смотрим его историю за две недели
до операции, на которую сработала система фрод-мониторинга.

In [24]:
hits0712

,event_id,user_id,rec_user_id,date_time,resolution,sub_channel
5,0fcea7fec4ff479cac8cf37b4555c817,75301207,49913000,2018-12-07 07:56:49,G,ATMAPI
6,c29a4e64d27b435b9b55aa3e62ce54d4,1200695,12797310,2018-12-07 07:14:41,G,MOBILEAPI
8,3107b2b1afcb490ab0a31135eb4b386c,45657780,23814017,2018-12-07 14:02:30,G,MOBILEAPI
10,d9bc95d991144d53b950d8084fa846bf,21683486,9865131,2018-12-07 10:32:41,G,WEBAPI
22,9fc959da4cd144ea8bf5bae2dc041a5e,4531894,VSP15472255,2018-12-07 18:22:52,G,MOBILEAPI
...,...,...,...,...,...,...
23682,28ea1a08c79240449922d5756646a353,92237374,69162645,2018-12-07 17:46:35,G,MOBILEAPI
23683,65a5b1dd74564e99b3b3c39b9ee460f7,3316389,89796796,2018-12-07 18:42:47,G,WEBAPI
23684,df808870913643c7ba24e990e6bbea39,24338601,58490342,2018-12-07 13:55:59,G,MOBILEAPI
23685,de1b30f52c264e95816e0cc163b2c116,2000352,37033043,2018-12-07 17:55:32,G,ATMAPI


`event_id` – внутренний id;<br>
`date_time` – время операции (GMT);<br>
`rec_user_id` – внутренний id клиента был получателем;<br>
`user_id` – внутренний id клиента, который отправлял.

In [25]:
print(*hits0712.columns.tolist())

event_id user_id rec_user_id date_time resolution sub_channel


In [26]:
hits0712.resolution.value_counts()

G    8943
F      63
S      10
Name: resolution, dtype: int64

За дроп будем считать клиента с резолюцией F (fraud) или S (suspicious).

Примерно 9000 ложно сработали и 73 нормально сработали на реальный фрод.

Наша задача – получить оценку для каждой транзакции, был ли rec_user_id дропом или легитимным
(именно для этого мы подтянули историю для отправителя и получателя).

Вытащим получателя и резолюцию.

In [27]:
target = hits0712[['rec_user_id', 'resolution']].drop_duplicates()
target.head()

,rec_user_id,resolution
5,49913000,G
6,12797310,G
8,23814017,G
10,9865131,G
22,VSP15472255,G


Целевыми классами мы считаем Fraud и Suspicious.

In [ ]:
target['class'] = target.resolution.map(lambda x: {'G': 0, 'F': 1, 'S': 1}[x])
target.head()

Получим примеры для обучения и контрольной выборки.

In [29]:
hist_trx.head()

,event_id,date_time,user_id,sub_channel,event_type,sub_type,atm_mcc,mcc_group,atm_merchant_name,amount
0,22c9f1ac686a43e18cdb798489193238,2018-12-06 09:33:21,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5921.0,R,YUG 426,280.0
1,62dac13fac68416d9bc340c51ddcb977,2018-12-06 09:36:08,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5331.0,R,MAGAZIN RODINA,376.9
2,9b666b8c9d4d4faea78e2b28a5468794,2018-12-06 12:21:02,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5921.0,R,YUG 426,143.0
3,d4d805fc3d5f4c91aa9b0389333b780c,2018-12-06 07:46:58,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5331.0,R,MAGAZIN RODINA,162.0
4,1d162351117a4418949dcf94111b9964,2018-12-06 12:21:27,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5331.0,R,MAGAZIN RODINA,56.0


In [30]:
new_data = hist_trx.copy()
new_data['one'] = (new_data['amount'] == 1.0).astype(int)
new_data.head()

,event_id,date_time,user_id,sub_channel,event_type,sub_type,atm_mcc,mcc_group,atm_merchant_name,amount,one
0,22c9f1ac686a43e18cdb798489193238,2018-12-06 09:33:21,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5921.0,R,YUG 426,280.0,0
1,62dac13fac68416d9bc340c51ddcb977,2018-12-06 09:36:08,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5331.0,R,MAGAZIN RODINA,376.9,0
2,9b666b8c9d4d4faea78e2b28a5468794,2018-12-06 12:21:02,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5921.0,R,YUG 426,143.0,0
3,d4d805fc3d5f4c91aa9b0389333b780c,2018-12-06 07:46:58,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5331.0,R,MAGAZIN RODINA,162.0,0
4,1d162351117a4418949dcf94111b9964,2018-12-06 12:21:27,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5331.0,R,MAGAZIN RODINA,56.0,0


Удаляем информацию за текущий день, чтобы не было утечек.

In [31]:
hist_trx_clean = hist_trx[hist_trx.date_time < '2018-12-07']
new_data_clean = new_data[new_data.date_time < '2018-12-07']

Объединяем очищенные данные с таргетом.

In [32]:
hist_trx_cleanest = pd.merge(
    hist_trx_clean,
    target,
    left_on='user_id',
    right_on='rec_user_id'
)
print(*hist_trx_cleanest.columns.to_list(), sep='\n')

event_id
date_time
user_id
sub_channel
event_type
sub_type
atm_mcc
mcc_group
atm_merchant_name
amount
rec_user_id
resolution
class


In [33]:
new_data_cleanest = pd.merge(
    new_data_clean,
    target,
    left_on ='user_id',
    right_on = 'rec_user_id'
)
print(*new_data_cleanest.columns.to_list(), sep='\n')

event_id
date_time
user_id
sub_channel
event_type
sub_type
atm_mcc
mcc_group
atm_merchant_name
amount
one
rec_user_id
resolution
class


In [34]:
hist_trx_cleanest[['user_id', 'mcc_group']].to_csv(
    'history_cleaned.csv',
    header=False
)
all_data = pd.read_csv(
    'history_cleaned.csv',
    header=0,
    names=['index', 'rec_user_id', 'mcc_group']
)

Есть получатель и информация, в каких точках он расплачивается.

In [35]:
all_data.head()

,index,rec_user_id,mcc_group
0,1,102050167,R
1,2,102050167,R
2,3,102050167,R
3,4,102050167,R
4,5,102050167,R


In [36]:
hist_trx_clean \
    .where(hist_trx_clean['user_id'] == '100013051') \
    .dropna()
hist_trx_clean \
    .groupby('user_id') \
    .apply(lambda x: np.std(x['amount'])) \
    .reset_index(name='count')

,user_id,count
0,100006366,436.417486
1,100013051,1380.384210
2,100017681,69139.844194
3,100023450,2335.039513
4,100031471,58.434334
...,...,...
14883,VSP847400,18095.500000
14884,VSP88139,24777.818727
14885,VSP9911909,413.317074
14886,VSP9912220,441.094041


In [37]:
hist_trx_clean \
    .groupby('user_id') \
    .apply(lambda x: (x['amount'] == 1).sum()) \
    .reset_index(name='count')

,user_id,count
0,100006366,1
1,100013051,7
2,100017681,21
3,100023450,0
4,100031471,1
...,...,...
14883,VSP847400,0
14884,VSP88139,0
14885,VSP9911909,0
14886,VSP9912220,0


Понимая характер трат и их количество, закодируем в 1 и 0 категориальные признаки.

In [38]:
all_data.mcc_group.value_counts()

R    86474
C    82975
U    52605
Z    31511
F    10366
X     2546
T     1182
A      660
H      354
J      238
Q      105
O       31
Name: mcc_group, dtype: int64

In [39]:
target_all = pd.merge(
    pd.pivot_table(
        all_data,
        index='rec_user_id',
        columns=['mcc_group'],
        values='index',
        aggfunc='count',
        fill_value=0
    ),
    target[['rec_user_id', 'class']],
    on='rec_user_id'
)

Новые данные:

In [40]:
new_data_cleanest[['user_id', 'mcc_group', 'one']].to_csv('history_cleaned_new_data.csv', header=False)
new_all_data = pd.read_csv(
    'history_cleaned_new_data.csv',
    header=0,
    names=['index', 'rec_user_id', 'mcc_group', 'one']
)

Есть получатель и информация, в каких точках он расплачивается.

In [41]:
new_all_data.head()

,index,rec_user_id,mcc_group,one
0,1,102050167,R,0
1,2,102050167,R,0
2,3,102050167,R,0
3,4,102050167,R,0
4,5,102050167,R,0


Понимая характер трат и их количество, закодируем в 1 и 0 категориальные признаки.

In [42]:
new_all_data.one.value_counts()

0    261144
1      8556
Name: one, dtype: int64

In [43]:
new_target_all = pd.merge(
    pd.pivot_table(
        new_all_data,
        index='rec_user_id',
        columns=['mcc_group'],
        values='index',
        aggfunc='count',
        fill_value=0
    ),
    target[['rec_user_id', 'class']],
    on='rec_user_id'
)
new_target_all['one'] = new_data_cleanest \
    .groupby('user_id') \
    .apply(lambda x: (x['amount'] == 1).sum()) \
    .reset_index(name='count')['count']
new_target_all['std'] = new_data_cleanest \
    .groupby('user_id') \
    .apply(lambda x: np.std(x['amount'])) \
    .reset_index(name='count')['count']
new_target_all = new_target_all.dropna()
new_target_all['one'] = new_target_all['one'].astype(int)

# получатель с классом 0 и где у него были траты
new_target_all

,rec_user_id,A,C,F,H,J,O,Q,R,T,U,X,Z,class,one,std
0,100017681,0,5,0,0,0,0,0,14,0,9,1,42,0,21,69139.844194
1,100023450,0,23,0,0,0,0,0,31,0,19,0,2,0,0,2335.039513
2,100031471,0,1,0,0,0,0,0,6,0,0,0,0,0,1,58.434334
3,10008487,0,24,6,0,0,0,0,21,0,41,1,7,0,1,220293.048516
4,10009395,0,6,0,0,0,0,0,13,0,16,0,0,0,0,16597.784888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6840,VSP4707035,0,3,0,0,0,0,0,10,0,0,0,3,0,3,24325.777545
6841,VSP4946126,0,0,0,0,0,0,0,2,0,3,0,9,0,3,17509.782306
6842,VSP4974530,0,1,10,0,0,0,0,29,0,20,2,0,0,0,758.450431
6843,VSP5027027,0,4,0,0,0,0,0,16,0,5,0,9,0,0,18095.500000


In [44]:
new_target_all['class'].value_counts()

0    6798
1      47
Name: class, dtype: int64

6798 – легитимные, 47 – фрод.

Получили данные:

In [45]:
x_data = new_target_all.drop(columns=['class', 'rec_user_id'])
y_data = new_target_all['class']

Разделяем выборку на обучающую и контрольную (20 процентов для тестирования):

In [46]:
x_train, x_test, y_train, y_test = train_test_split(
    x_data,
    y_data,
    test_size=.2,
    random_state=123
)

Количество записей:

In [47]:
x_test.shape

(1369, 14)

In [48]:
y_train.shape

(5476,)

Моделирование (можно менять гиперпараметры, чтобы избавиться от дизбаланса):

In [49]:
clf = RandomForestClassifier(class_weight={0: 1, 1: 15})
clf.fit(x_train, y_train)

y_pred = pd.Series(clf.predict(x_test), index=x_test.index)

y_test.value_counts()

0    1362
1       7
Name: class, dtype: int64

Результат работы модели:

In [50]:
confusion_matrix(y_test, y_pred)

array([[1361,    1],
       [   7,    0]], dtype=int64)

In [51]:
print(classification_report(y_test, y_pred, target_names=['Genuine', 'Fraud']))

              precision    recall  f1-score   support

     Genuine       0.99      1.00      1.00      1362
       Fraud       0.00      0.00      0.00         7

    accuracy                           0.99      1369
   macro avg       0.50      0.50      0.50      1369
weighted avg       0.99      0.99      0.99      1369

